In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

# Exercises

1. Read the case, department, and source data into their own spark dataframes.

In [2]:
# specify the data types before creating df
from pyspark.sql.types import StructType, StructField, StringType

schema_src = StructType([
    StructField('source_id', StringType()),
    StructField('source_username', StringType()),
])

In [3]:
# reading data to a spark dataframe
src = spark.read.csv("data/source.csv", sep=",", header=True, schema=schema_src)
dep = spark.read.csv("data/dept.csv", sep=",", header=True, inferSchema=True)
case = spark.read.csv("data/case.csv", sep=",", header=True, inferSchema=True)

In [4]:
# looking into first two rows of data
case.show(1, vertical=True), dep.show(1, vertical=True), src.show(3)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row

+---------+----------------+
|source_id| s

(None, None, None)

2. Let's see how writing to the local disk works in spark:

    - Write the code necessary to store the source data in both csv and json format, store these as `sources_csv` and `sources_json`
    - Inspect your folder structure. What do you notice?

In [5]:
# write data to csv locally
#src.write.csv('sources-csv')

In [6]:
# write data to json locally
#src.write.json('sources-json')

In [7]:
# there is a _SUCCESS folder and another folder with the unique content for each file type

3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [8]:
src.printSchema(), dep.printSchema(), case.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



(None, None, None)

##### Case data:
- `case_id` --> string, since it's indentifier
- `case_opened_date`, `case_closed_data`, `SLA_due_date` --> date types
- `case_late` and `case_closed` --> boolean

In [9]:
case = case.withColumn('case_id', col('case_id').cast('string'))

In [10]:
# rename SLA_due_date to case_due_date
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

fmt = "M/d/yy H:mm"
case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_due_date", fmt))
)

In [11]:
case = (case
    .withColumn('case_late', col('case_late').cast('boolean'))
    .withColumn('case_closed', col('case_closed').cast('boolean'))
)

In [12]:
case.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



---

1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [64]:
# oldest past due
(
    case
    .select('*',
        datediff(current_timestamp(),'case_due_date').alias('days_past_due'))
    .filter(col('case_closed') == 'false')
    .sort(desc('days_past_due'))
    .show(1, vertical=True)
)

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | null                 
 case_due_date        | 2017-01-17 08:30:00  
 case_late            | true                 
 num_days_late        | 348.6458333          
 case_closed          | false                
 service_request_type | No Address Posted    
 SLA_days             | 15.77859954          
 case_status          | Open                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
 department           | DSD/Code Enforcement 
 dept_subject_to_SLA  | true                 
 days_past_due        | 1414                 
only showing top 1 row



In [65]:
# oldest current case since opening
(
    case
    .select('*',
        datediff(current_timestamp(),'case_opened_date').alias('case_age'))
    .filter(col('case_closed') == 'false')
    .sort(desc('case_age'))
    .show(1, vertical=True)
)

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | null                 
 case_due_date        | 2017-01-17 08:30:00  
 case_late            | true                 
 num_days_late        | 348.6458333          
 case_closed          | false                
 service_request_type | No Address Posted    
 SLA_days             | 15.77859954          
 case_status          | Open                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
 department           | DSD/Code Enforcement 
 dept_subject_to_SLA  | true                 
 case_age             | 1430                 
only showing top 1 row



2. How many Stray Animal cases are there?

In [15]:
case.filter(col('service_request_type') == 'Stray Animal').count()

26760

3. How many service requests that are assigned to the Field Operations department (`dept_division`) are not classified as "Officer Standby" request type (`service_request_type`)?

In [16]:
(
    case
    .filter(col('dept_division') == 'Field Operations')
    .filter(col('service_request_type') != 'Officer Standby')
    .count()
)

113902

4. Convert the `council_district` column to a string column.

In [69]:
case = case.withColumn("council_district", col("council_district").cast("string"))

5. Extract the year from the case_closed_date column.

In [18]:
(
    case
    .withColumn('year', year('case_closed_date'))
    .select('year', 'case_closed_date')
    .show(5)
)

+----+-------------------+
|year|   case_closed_date|
+----+-------------------+
|2018|2018-01-01 12:29:00|
|2018|2018-01-03 08:11:00|
|2018|2018-01-02 07:57:00|
|2018|2018-01-02 08:13:00|
|2018|2018-01-01 13:29:00|
+----+-------------------+
only showing top 5 rows



6. Convert num_days_late from days to hours in new columns num_hours_late.

In [70]:
(
    case
    .withColumn('num_hours_late', expr('num_days_late * 24'))
    .select('num_days_late', 'num_hours_late')
    .show(5)
)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
+-------------------+-------------------+
only showing top 5 rows



7. Join the case data with the source and department data.

In [20]:
# join dep to case
case = (
    case
    # left join on dept_division
    .join(dep, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dep.dept_division)
    .drop(dep.dept_name)
    .drop(case.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

In [71]:
# join source to case
df = (
    case
    # left join on source_id
    .join(src, "source_id", "left")
    # drop source_id from src
    .drop(src.source_id)
)

df.show(1, vertical=True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 source_username      | svcCRMLS             
only showing top 1 row



8. Are there any cases that do not have a request source?

In [35]:
df = df.withColumn("request_address", trim(lower(df.request_address)))

In [45]:
df.filter(df.request_address.isNull()).show(vertical=True)

(0 rows)



9. What are the top 10 service request types in terms of number of requests?

In [33]:
(
    df.groupBy('service_request_type')
    .count()
    .sort(desc('count'))
    .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



10. What are the top 10 service request types in terms of average days late?

In [84]:
(
    df.where('case_late')
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('avg_days_late'))
    .sort(desc('avg_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+
|service_request_type                  |avg_days_late     |
+--------------------------------------+------------------+
|Zoning: Recycle Yard                  |210.89201994318182|
|Zoning: Junk Yards                    |200.20517608494276|
|Structure/Housing Maintenance         |190.20707698509804|
|Donation Container Enforcement        |171.09115313942618|
|Storage of Used Mattress              |163.96812829714287|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Traffic Signal Graffiti               |137.64583330000002|
|License Requied Used Mattress Sales   |128.79828704142858|
+--------------------------------------+------------------+
only showing top 10 rows



11. Does number of days late depend on department?

In [87]:
( 
    df.where('case_late')
    .groupBy('department')
    .agg(avg('num_days_late').alias('avg_days_late'))
    .sort(desc('avg_days_late'))
    .show()
)

+--------------------+------------------+
|          department|     avg_days_late|
+--------------------+------------------+
|    Customer Service| 88.18248182589824|
|DSD/Code Enforcement| 49.50633998635033|
|Animal Care Services| 23.44672963473822|
|  Parks & Recreation|22.427807192724128|
|Trans & Cap Impro...| 10.66295045507867|
|         Solid Waste| 7.147172789557422|
|        Metro Health| 6.494699602827868|
+--------------------+------------------+



12. How do number of days late depend on department and request type?

In [88]:
( 
    df.where('case_late')
    .where('case_closed')
    .groupBy('department', 'service_request_type')
    .agg(avg('num_days_late').alias('avg_days_late'))
    .sort(desc('avg_days_late'))
    .show()
)

+--------------------+--------------------+------------------+
|          department|service_request_type|     avg_days_late|
+--------------------+--------------------+------------------+
|DSD/Code Enforcement|Zoning: Recycle Yard| 273.6008813301333|
|DSD/Code Enforcement|  Zoning: Junk Yards| 251.9073916303836|
|DSD/Code Enforcement|Donation Containe...|201.74241051817063|
|DSD/Code Enforcement|Structure/Housing...|182.42093904966666|
|DSD/Code Enforcement|Graffiti: Private...| 175.0883448666667|
|DSD/Code Enforcement|Storage of Used M...|163.96812829714287|
|DSD/Code Enforcement|Labeling for Used...|162.43032902285717|
|DSD/Code Enforcement|Record Keeping of...|153.99724039428568|
|DSD/Code Enforcement|Signage Requied f...|151.63868055333333|
|DSD/Code Enforcement|License Requied U...|128.79828704142858|
|DSD/Code Enforcement|             Vendors|127.33261922710678|
|DSD/Code Enforcement|         Curbstoning|121.75948969483001|
|DSD/Code Enforcement|DSDB Ordered/Hold...|119.80444964

**Bonus**: You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.

In [79]:
# latest date is 'max due date'
(
df.select(max(col('case_opened_date'))).show(), 
df.select(max(col('case_closed_date'))).show(), 
df.select(max(col('case_due_date'))).show()
)

+---------------------+
|max(case_opened_date)|
+---------------------+
|  2018-08-08 10:38:00|
+---------------------+

+---------------------+
|max(case_closed_date)|
+---------------------+
|  2018-08-08 10:38:00|
+---------------------+

+-------------------+
| max(case_due_date)|
+-------------------+
|2022-05-27 15:07:00|
+-------------------+



(None, None, None)

In [82]:
df.filter(col('case_due_date') >= current_timestamp()).show(vertical=True)

-RECORD 0------------------------------------
 source_id            | CRM_Listener         
 case_id              | 1014178781           
 case_opened_date     | 2018-01-19 17:07:00  
 case_closed_date     | null                 
 case_due_date        | 2021-11-25 08:30:00  
 case_late            | false                
 num_days_late        | -1204.354167         
 case_closed          | false                
 service_request_type | Signal Timing Mod... 
 SLA_days             | 1405.640856          
 case_status          | Open                 
 request_address      | MACARTHUR VIEW an... 
 council_district     | 10                   
 department           | Trans & Cap Impro... 
 dept_subject_to_SLA  | true                 
 source_username      | CRM_Listener         
-RECORD 1------------------------------------
 source_id            | CRM_Listener         
 case_id              | 1014250897           
 case_opened_date     | 2018-02-16 16:42:00  
 case_closed_date     | null      